# Chapter 6
## 6.1 Case Study : Refactoring Strategy
### Classic Strategy

In [17]:
# Example 6-1 : Implementation Order class with pluggable discount strategies
from abc import ABC, abstractmethod
from collections import namedtuple

Customer = namedtuple('customer', 'name fidelity')

class LineItem:
    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price
        
    def total(self):
        return self.price * self. quantity
    
class Order:
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion
        
    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
            
        return self.__total
    
    def due(self):
        if self.promotion is None:
            discount=0
        else:
            discount = self.promotion.discount(self)
            
        return self.total() - discount
    
    def __repr__(self):
        fmt = '<Order total: {:.2f} due:{:.2f}>'
        return fmt.format(self.total(), self.due())
    
class Promotion(ABC):
    @abstractmethod
    def discount(self, order):
        """Return discount as a positive dollar amount"""
        
class FidelityPromo(Promotion):
    """5% discount for customers with 1000 or more fidelity points"""
    
    def discount(self, order):
        return order.total()*0.05 if order.customer.fidelity >= 1000 else 0
    
class BulkItemPromo(Promotion):
    """10% discount for each LineItem with 20 or more units"""
    
    def discount(self, order):
        discount = 0
        for item in order.cart:
            if item.quantity >= 20:
                discount += item.total*.1
        return discount
    
class LargeOrderPromo(Promotion):
    """7% discount for orders with 10 or more distinct items"""
    
    def discount(self, order):
        distinct_items = {item.product for item in order.cart}
        
        if len(distinct_items) >= 10:
            return order.total()*.07
        return 0

In [18]:
# Example 6-2 : Sample usage of Order class with different promotions applied
joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)
cart = [LineItem('banana', 4, .5),
        LineItem('apple', 10, 1.5),
        LineItem('watermellon', 5, 5.0)]
Order(joe, cart, FidelityPromo())




<Order total: 42.00 due:42.00>

### Function-Oriented Strategy

In [19]:
# Example 6-3 : Order class with discount strategies implemented as functions

from collections import namedtuple

Customer = namedtuple('Customer', 'name fidelity')

class LineItem:
    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price
        
    def total(self):
        return self.price*self.quantity
    
class Order: # the Context
    def __init__(self, customer, cart, promotion = None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion
        
    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total
    
    def due(self):
        if self.promotion is None:
            discount = 0 
        else: 
            discount = self.promotion(self)
        return self.total()-discount
    
    def __repr__(self):
        fmt = '<Order total: {:.2f} due:{:.2f}>'
        return fmt.format(self.total(), self.due())
def fidelity_promo(order):
    """5% discount for customers with 1000 or more fidelity points"""
    return order.total()*.05 if order.customer.fidelity >= 1000 else 0

def bulk_item_promo(order):
    """10% discount for each LineItem with 20 or more units"""
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total()*.1
    return discount

def large_order_promo(order):
    """7% discount for orders with 10 or more distinct items"""
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total()

### Choosing the Best Strategy : Simple Approach

In [21]:
# Example 6-6: best_promo finds the maximum discount iterating over a list of functions
promos = [fidelity_promo, bulk_item_promo, large_order_promo]

def best_promo(order):
    """Select best discount available"""
    return max(promo(order) for promo in promos)


### Finding Strategies in a Module

In [25]:
# Example 6-7 : The promos list is built by introspection of the module global namespace
promos = [globals()[name] for name in globals()
          if name.endswith('_promo')
         and name != 'best_promo']
def best_promo(order):
    """Select best discount available"""
    return max(promo(order) for promo in promos)

In [26]:
# Example 6-8 : The promos list is built by introspection of a new promotions module

promos = [func for name, func in inspect.getmembers(promotions, inspect.isfunction)]

def best_promo(order):
    """Select best discount available"""
    
    return max(promo(order) for promo in promos)

NameError: name 'inspect' is not defined